In [79]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [80]:
import os
os.chdir("/kaggle/input/ammi-bootcamp-kaggle-competition/")
os.getcwd()

In [81]:
import cudf as cd
import cupy as cp
import cuml as cm
import pandas as pd
import numpy as np
trainwine= pd.read_csv('train.csv')
testwine = pd.read_csv('test.csv')

In [82]:
trainwine.set_index('id',inplace=True)
testwine.set_index('id',inplace=True)

In [83]:
#convert points data type from float to integer
trainwine['points'] = trainwine['points'].astype(int)
testwine['points'] = testwine['points'].astype(int)

In [84]:
for i in trainwine.columns:
  if trainwine[i].dtype == 'O':
    trainwine[i] = trainwine[i].str.lower()
    
for i in testwine.columns:
  if testwine[i].dtype == 'O':
    testwine[i] = testwine[i].str.lower()  

In [85]:
p = ['title','taster_twitter_handle','taster_name']
q = ['index','title','taster_twitter_handle','taster_name','price']
trainwine.drop(columns=p, axis =1,inplace=True)
testwine.drop(columns=q, axis =1,inplace=True)

In [86]:
trainwine.head()

In [87]:
testwine.head()

In [88]:
def transform_points(points):
    if points < 84:
        return 1
    elif points >= 84 and points < 89:
        return 2 
    elif points >= 89 and points < 95:
        return 3 
    elif points >= 95 and points < 99:
        return 4 
    else:
        return 5

In [89]:
trainwine = trainwine.assign(points_cat = trainwine['points'].apply(transform_points))
testwine = testwine.assign(points_cat = testwine['points'].apply(transform_points))

In [90]:
trainwine.isnull().sum().sort_values(ascending=False)

In [91]:
testwine.isnull().sum().sort_values(ascending=False)

In [92]:
trainwine.drop('region_2',axis=1,inplace=True)
testwine.drop('region_2',axis=1,inplace=True)

In [93]:
for i in trainwine.columns:
  if trainwine[i].isnull().sum() >= 1:
    trainwine[i].fillna('missing',inplace = True)

In [94]:
for i in testwine.columns:
  if testwine[i].isnull().sum() >= 1:
    testwine[i].fillna('missing',inplace = True)

In [95]:
import matplotlib.pyplot as plt 
import seaborn as sb
plt.figure(figsize=(22,7))
sb.boxplot(data=trainwine,x='points_cat',y='price')
plt.xlabel('Points')
plt.ylabel('Price of wine')
plt.title('Regression plot showing the relationship between points and price of wine.')
plt.show()

In [97]:
catfeat = ['country', 'designation', 'province','region_1', 'variety', 'winery']
encoder_dict = {}
for i in catfeat:
  t = trainwine.groupby([i])['price'].mean().sort_values(ascending=True).index
  encoder_dict[i] = {k:i for i,k in enumerate(t,0)}

for i in catfeat:
  trainwine[i] = trainwine[i].map(encoder_dict[i])
  testwine[i] = testwine[i].map(encoder_dict[i])

In [98]:
testwine.fillna(testwine.median(),inplace=True)

In [103]:
from cuml.model_selection import train_test_split
x = cd.from_pandas(trainwine.drop(['price','description','points_cat'], axis=1))
y = cd.from_pandas(trainwine['price'])
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.2, random_state=42)

In [108]:
from cuml.linear_model import LinearRegression
from cuml.metrics import mean_squared_error
lin = LinearRegression()
lin = lin.fit(xtrain.astype(float),ytrain.astype(float))
linpred = lin.predict(xtest)
rmse = np.sqrt(mean_squared_error(ytest,linpred))
print('The RMSE for the linear regression model is {}.'.format(rmse))